<a href="https://colab.research.google.com/github/shawn-kg/DataMining/blob/main/assign8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Preprocessing

In [29]:
import numpy as np
import pandas as pd

data = pd.read_csv('./codon_usage.csv')
data = data.drop(data.columns[[1,2,3,4]], axis=1)
data = data.to_numpy()
data = np.delete(data,[486,5063],0)
target = data[:,0]
data = np.delete(data,0,1)
data = data.astype(float)
a=_, b = np.unique(target, return_index = True)
classes = target[np.sort(b)]
target_dict = dict.fromkeys(target[np.sort(b)], 0)
for i in target:
  target_dict[i]+=1
true_sizes = []

ht = 0  
for i in target_dict.keys():
  ht += (target_dict[i]/data.shape[0])*np.log(target_dict[i]/data.shape[0])
  true_sizes.append(target_dict[i])
target_ind = [0,0,0,0,0,0,0,0,0,0,0]

j=0
for i in target_dict.keys():
  if j!=0:
    target_ind[j]+=target_dict[i] + target_ind[j-1]
  else:
    target_ind[j] = target_dict[i]
  j+=1

ht = -ht


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


##Expectation Maximization Clustering

In [30]:
from pyparsing import javaStyleComment
from prompt_toolkit.key_binding.bindings.emacs import has_selection
from scipy.stats import multivariate_normal
import scipy.special
from sklearn.metrics.cluster import normalized_mutual_info_score
import math

np.random.seed(42)
def pij(k, target_ind, cluster_index, n):
  pijs = np.zeros((k,k))
  for c in range(k):
    beg = 0
    end = 0
    for d in range(k):
      count = 0
      if d!=0:
        beg = target_ind[d-1]
      else:
        beg = 0
      end = target_ind[d]
      for ind in range(len(cluster_index[c])):
        if (cluster_index[c][ind] >= beg and cluster_index[c][ind] <end):
          count+=1
      pijs[c][d] = count/n
  hct = 0 
  for i in range(k):
    for j in range(k):
      hct += pijs[i][j] * np.log(pijs[i][j] + (.0000000000001))
  hct = -hct
  return hct


def HC(final_clusters, k, n):
  hc = 0
  for h in range(k):
    p = len(final_clusters[h])/n
    if p!=0:
      hc += p * np.log(p)
  hc = -hc
  return hc
def find_clusters(D, k, cluster_means):
  clusters = dict()
  clusters_index = dict()
  for n in range(k):
    clusters[n] = []
    clusters_index[n] = []
  for j in range(D.shape[0]):
    dists = np.zeros((k))
    for i in range(k):
      distance = np.linalg.norm(D[j] - cluster_means[i])**2
      dists[i] = distance
    clusters[np.argmin(dists)].append(D[j])
    clusters_index[np.argmin(dists)].append(j)
  for n in range(k):
    clusters[n] = np.array(clusters[n])
  return clusters, clusters_index

def getCov(clusters, k):
  covMatrices = dict()
  for i in range(k):
    means = np.mean(clusters[i], axis=0)
    xHat = clusters[i]-means
    cov = np.dot(np.transpose(xHat),xHat)/xHat.shape[0]
    covMatrices[i] = cov
  return covMatrices

def EMC(D, k, eps, ht):
  n = D.shape[0]
  d = D.shape[1]
  ind = np.random.choice(range(n), k, replace=False)
  cluster_means = np.copy(D[ind])
  clusters, _ = find_clusters(D,k,cluster_means)
  covMatrices = getCov(clusters,k)
  probs = np.zeros((k))
  for i in range(k):
    probs[i] = clusters[i].shape[0]/n
  #Loop
  while (True):
    prev_means = np.copy(cluster_means)
    
    # Expectation Step
    weights = np.zeros((k,n))
    logweights = np.zeros((k,n))
    for i in range(k):
      logweights[i] = multivariate_normal.logpdf(D, cluster_means[i], covMatrices[i], allow_singular=True) + np.log(probs[i])
    sums = scipy.special.logsumexp(logweights, axis=0)
    weights = np.exp(logweights - sums)
    
    # Maximization Step
    for i in range(k):
      #Means
      weightsums = weights[i] @ D
      cluster_means[i] = weightsums/np.sum(weights[i])
      # covariances
      cov = np.zeros((d,d))
      for j in range(n):
        dHat = (D[j] - cluster_means[i]).reshape((-1,1))
        cov += weights[i][j]*np.dot(dHat,np.transpose(dHat))
      covMatrices[i] = cov/np.sum(weights[i])
      # probs
      probs[i] = np.sum(weights[i])/n
    
    # convergence check
    means_sum = 0
    for i in range(k):
      temp = cluster_means[i] - prev_means[i]
      means_sum += np.linalg.norm(temp)**2
    if means_sum<eps:
      break
  
  # Clusters
  final_clusters = dict()
  for i in range(k):
    final_clusters[i] = []
  for j in range(n):
    final_clusters[np.argmax(weights[:,j])].append(j)
  

  hc = HC(final_clusters,k,n)

  hct = pij(k, target_ind, final_clusters, n)

  ict = hc + ht - hct
  nmi = ict/math.sqrt(hc*ht)

  return cluster_means, covMatrices, final_clusters, nmi

finalMeans = np.zeros((11,64)) 
finalCovMatrices = dict()
finalClusters = dict()
final_nmi = 0
for iter in range(10):
  means, covMatrices, clusters, nmi = EMC(data, 11, .001, ht)
  if nmi>final_nmi:
    final_nmi = nmi
    finalClusters = clusters
    finalCovMatrices = covMatrices
    finalMeans = means
for i in range(11):
  print("Cluster", i, ":")
  print("Mean: ")
  print(finalMeans[i])
  print("Covariance Matrix: ")
  print(covMatrices[i])
  print("Size:", len(finalClusters[i]))
print("NMI:", final_nmi)

Cluster 0 :
Mean: 
[0.01063693 0.02553401 0.00290686 0.01191388 0.00978071 0.02262803
 0.00306359 0.04434053 0.01251909 0.03299371 0.0030524  0.02427678
 0.0105223  0.02639077 0.00548588 0.0302754  0.01278103 0.04344525
 0.01393767 0.03616991 0.006192   0.01373513 0.00567852 0.02491547
 0.01484813 0.01488306 0.04345034 0.00876261 0.01253537 0.00461673
 0.01270236 0.00458121 0.01589102 0.00469726 0.01688176 0.00536005
 0.02619915 0.00568675 0.01829317 0.00984098 0.01746841 0.00907794
 0.02791241 0.00964972 0.02289238 0.00270955 0.00868311 0.00890134
 0.01439816 0.01358282 0.02779736 0.01081336 0.03015576 0.00495605
 0.0129453  0.00234739 0.00385008 0.02141244 0.03587621 0.02577867
 0.03325994 0.00098961 0.00050032 0.0016385 ]
Covariance Matrix: 
[[ 1.01407011e-04 -2.62362045e-05  6.87242139e-05 ...  4.45102993e-06
   2.53371688e-06  3.20364754e-05]
 [-2.62362045e-05  1.02140777e-04 -4.49708704e-05 ...  8.64939880e-07
  -7.91978110e-07 -1.47217490e-06]
 [ 6.87242139e-05 -4.49708704e-05  

##K-Means Clustering

In [31]:
def K_Means(D, k, eps, ht):
  ind = np.random.choice(range(D.shape[0]), k, replace=False)
  cluster_means = D[ind]
  t=0
  while (True):
    t=t+1
    clusters, clusters_index = find_clusters(D,k,cluster_means)
    new_means = np.zeros((k,D.shape[1]))
    for i in range(k):
      new_means[i] = (1/clusters[i].shape[0])*np.sum(clusters[i],axis=0)
    means_sum = 0
    for i in range(k):
      temp = cluster_means[i] - new_means[i]
      means_sum += np.linalg.norm(temp)**2
    cluster_means = np.copy(new_means)
    if means_sum<=eps:
      break
  
  hc = HC(clusters, k, D.shape[0])
  hct = pij(k, target_ind, clusters_index, D.shape[0])
  ict = hc + ht - hct
  nmi = ict/math.sqrt(hc*ht)
  return cluster_means, clusters, nmi


finalMeans = np.zeros((11,64)) 
finalCovMatrices = dict()
finalClusters = dict()
final_nmi = 0
for iter in range(10):
  cluster_means, clusters, nmi = K_Means(data, 11, .001, ht)
  if nmi>final_nmi:
    final_nmi = nmi
    finalClusters = clusters
    finalCovMatrices = covMatrices
    finalMeans = means

for i in range(11):
  print("Cluster ", i, ":")
  print("Mean Vector")
  print(finalMeans[i])
  print("Size")
  print(finalClusters[i].shape[0])
print("NMI:", final_nmi)

Cluster  0 :
Mean Vector
[0.02655659 0.01608412 0.02706957 0.01837475 0.01663066 0.00904595
 0.01290197 0.01713938 0.03879694 0.02116808 0.006746   0.02603312
 0.02559876 0.01200873 0.0165155  0.0186804  0.0290065  0.01991245
 0.02492644 0.01820897 0.01318582 0.0063752  0.01419975 0.00847468
 0.01188807 0.03532904 0.02104294 0.01083146 0.00916348 0.01536429
 0.0063956  0.01279938 0.00640186 0.01063082 0.01102863 0.01870724
 0.01694471 0.01464189 0.00988768 0.01958769 0.01192912 0.02672661
 0.01455912 0.02402628 0.02042862 0.00534032 0.0034026  0.01132371
 0.00808495 0.03378065 0.01533929 0.01853241 0.01098001 0.00438661
 0.00373469 0.00369748 0.00131235 0.03310259 0.01665156 0.03655463
 0.0161012  0.00228161 0.00049306 0.0029438 ]
Size
618
Cluster  1 :
Mean Vector
[0.0132514  0.02775501 0.00397229 0.01303616 0.01208639 0.0221277
 0.00444435 0.03179053 0.01500073 0.02958055 0.0056022  0.02475397
 0.01379907 0.02386104 0.00654492 0.02536955 0.0189446  0.03443348
 0.01393528 0.01901571 0.